# DeepHit for Competing Risks

This notebook implements **DeepHit** (Lee et al., 2018) for competing risks analysis. DeepHit is a deep learning approach that directly models the probability mass function of survival times without assuming any particular form for the underlying stochastic process.

## Methodology

| Aspect | Description |
|--------|-------------|
| **Model** | DeepHit with competing risks |
| **Base** | pycox DeepHitSingle / DeepHit |
| **Features** | Blumenstock et al. (2022) - 21 variables |
| **Evaluation** | Time-dependent C-index at 24, 48, 72 months |

## Key Advantages of DeepHit

| Feature | Description |
|---------|-------------|
| **No PH assumption** | Does not assume proportional hazards |
| **Competing risks** | Native support for multiple event types |
| **Flexible** | Can capture complex non-linear relationships |
| **Direct modeling** | Models PMF directly, not hazard function |

## References

- Lee, C., Zame, W., Yoon, J., & van der Schaar, M. (2018). DeepHit: A Deep Learning Approach to Survival Analysis with Competing Risks. AAAI.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Deep learning
import torch
import torchtuples as tt
from pycox.models import DeepHitSingle
from pycox.evaluation import EvalSurv

# Survival analysis (for comparison metrics)
from sksurv.util import Surv
from sksurv.metrics import concordance_index_censored, concordance_index_ipcw

# Preprocessing
from sklearn.preprocessing import StandardScaler
import pickle

sns.set_style('whitegrid')
%matplotlib inline

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Time horizons for evaluation (matching previous notebooks)
TIME_HORIZONS = [24, 48, 72]

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Time horizons for C-index evaluation: {TIME_HORIZONS} months")

In [ ]:
# === CONFIGURATION ===
DATA_DIR = Path('../data/processed')
FIGURES_DIR = Path('../reports/figures')
MODELS_DIR = Path('../models')

FIGURES_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

# Cross-validation folds (Blumenstock methodology)
TRAIN_FOLDS = list(range(10))  # Folds 0-9 for training
VAL_FOLDS = [9]                 # Use fold 9 for validation (early stopping)
TEST_FOLD = 10                  # Fold 10 for testing

# DeepHit hyperparameters
DEEPHIT_PARAMS = {
    'num_nodes': [64, 64],      # Hidden layer sizes
    'batch_norm': True,
    'dropout': 0.2,
    'activation': torch.nn.ReLU,
    'batch_size': 256,
    'epochs': 100,
    'learning_rate': 0.01,
    'alpha': 0.2,               # Weight for ranking loss
    'sigma': 0.1,               # Smoothing parameter for ranking loss
}

# Number of discrete time points
NUM_DURATIONS = 100

print(f"Training folds: {[f for f in TRAIN_FOLDS if f not in VAL_FOLDS]}")
print(f"Validation fold: {VAL_FOLDS}")
print(f"Test fold: {TEST_FOLD}")
print(f"\nDeepHit parameters:")
for k, v in DEEPHIT_PARAMS.items():
    print(f"  {k}: {v}")

---

## Load Loan-Month Panel Data

In [ ]:
# Load the loan-month panel data
print("Loading loan-month panel data...")
panel_df = pd.read_parquet(DATA_DIR / 'loan_month_panel.parquet')

print(f"Loaded {len(panel_df):,} loan-months")
print(f"Unique loans: {panel_df['loan_sequence_number'].nunique():,}")
print(f"Folds: {sorted(panel_df['fold'].unique())}")
print(f"Vintages: {panel_df['vintage_year'].min()} - {panel_df['vintage_year'].max()}")

print("\nEvent distribution (terminal observations):")
event_names = {0: 'Censored', 1: 'Prepay', 2: 'Default'}
terminal_events = panel_df[panel_df['event'] == 1].groupby('event_code').size()
for code, count in terminal_events.items():
    print(f"  {event_names.get(code, 'Other')} (k={code}): {count:,}")

---

## Define Features (Blumenstock et al. 2022)

In [ ]:
# Define feature groups (Blumenstock et al. 2022, Table 2)
# Matching previous notebooks exactly

# Static covariates (fixed at origination) - 5 variables
STATIC_FEATURES = [
    'int_rate',      # Initial interest rate
    'orig_upb',      # Original unpaid balance
    'fico_score',    # Initial FICO score
    'dti_r',         # Initial debt-to-income ratio
    'ltv_r',         # Initial loan-to-value ratio
]

# Behavioral covariates (time-varying) - 4 variables
BEHAVIORAL_FEATURES = [
    'bal_repaid',      # Current repaid balance in percent
    't_act_12m',       # No. of times not being delinquent in last 12 months
    't_del_30d_12m',   # No. of times being 30 days delinquent in last 12 months
    't_del_60d_12m',   # No. of times being 60 days delinquent in last 12 months
]

# Macro covariates (time-varying) - 12 variables
MACRO_FEATURES = [
    'hpi_st_d_t_o',    # HPI difference between origination and today (state)
    'ppi_c_FRMA',      # Current prepayment incentive
    'TB10Y_d_t_o',     # Treasury rate difference
    'FRMA30Y_d_t_o',   # 30Y FRM difference
    'ppi_o_FRMA',      # Prepayment incentive at origination
    'hpi_st_log12m',   # HPI 12-month log return (state)
    'hpi_r_st_us',     # Ratio of state HPI to national HPI
    'st_unemp_r12m',   # Unemployment 12-month log return (state)
    'st_unemp_r3m',    # Unemployment 3-month log return (state)
    'TB10Y_r12m',      # Treasury rate 12-month return
    'T10Y3MM',         # Yield spread (10Y - 3M)
    'T10Y3MM_r12m',    # Yield spread 12-month return
]

ALL_FEATURES = STATIC_FEATURES + BEHAVIORAL_FEATURES + MACRO_FEATURES

# Filter to available features
feature_cols = [f for f in ALL_FEATURES if f in panel_df.columns]
missing_features = [f for f in ALL_FEATURES if f not in panel_df.columns]

print("=== Feature Groups (Blumenstock et al. 2022) ===")
print(f"Static features: {len([f for f in STATIC_FEATURES if f in feature_cols])}/5")
print(f"Behavioral features: {len([f for f in BEHAVIORAL_FEATURES if f in feature_cols])}/4")
print(f"Macro features: {len([f for f in MACRO_FEATURES if f in feature_cols])}/12")
print(f"\nTotal available: {len(feature_cols)}/21")

if missing_features:
    print(f"\nMissing features ({len(missing_features)}):")
    for f in missing_features:
        print(f"  - {f}")

---

## Prepare Data for DeepHit

DeepHit requires terminal observations (one per loan) with discretized time.

In [ ]:
# For DeepHit, we need terminal observations (one per loan)
# Get the last observation for each loan
print("=== Preparing Terminal Observations ===")

time_col = 'loan_age'
event_col = 'event_code'

# Sort and get last observation per loan
panel_df = panel_df.sort_values(['loan_sequence_number', time_col])
terminal_df = panel_df.groupby('loan_sequence_number').last().reset_index()

print(f"Terminal observations: {len(terminal_df):,} loans")

# Lag bal_repaid to avoid data leakage (matching previous notebooks)
if 'bal_repaid' in feature_cols:
    print("\nLagging bal_repaid to avoid data leakage...")
    
    def get_lagged_bal_repaid(group):
        if len(group) >= 2:
            return group['bal_repaid'].iloc[-2]
        else:
            return group['bal_repaid'].iloc[-1]
    
    bal_repaid_lag = panel_df.groupby('loan_sequence_number').apply(get_lagged_bal_repaid)
    terminal_df['bal_repaid_lag1'] = terminal_df['loan_sequence_number'].map(bal_repaid_lag)
    
    feature_cols = [f if f != 'bal_repaid' else 'bal_repaid_lag1' for f in feature_cols]
    print("  Created bal_repaid_lag1")

# Log transform UPB
if 'orig_upb' in terminal_df.columns:
    terminal_df['log_upb'] = np.log(terminal_df['orig_upb'])
    feature_cols = [f if f != 'orig_upb' else 'log_upb' for f in feature_cols]
    print("  Created log_upb")

# Drop rows with missing features
n_before = len(terminal_df)
terminal_df = terminal_df.dropna(subset=feature_cols)
n_after = len(terminal_df)
print(f"\nAfter dropping NaN: {n_after:,} loans (dropped {n_before - n_after:,})")

print(f"\nFeatures ({len(feature_cols)}): {feature_cols}")

In [ ]:
# Split by folds (matching previous notebooks)
print("=== Splitting Data by Folds ===")

# Training uses folds 0-8, validation uses fold 9, test uses fold 10
train_folds_actual = [f for f in TRAIN_FOLDS if f not in VAL_FOLDS]

train_df = terminal_df[terminal_df['fold'].isin(train_folds_actual)].copy()
val_df = terminal_df[terminal_df['fold'].isin(VAL_FOLDS)].copy()
test_df = terminal_df[terminal_df['fold'] == TEST_FOLD].copy()

print(f"Training set (folds {train_folds_actual}): {len(train_df):,} loans")
print(f"Validation set (fold {VAL_FOLDS}): {len(val_df):,} loans")
print(f"Test set (fold {TEST_FOLD}): {len(test_df):,} loans")

# Event distribution
print("\nTraining set event distribution:")
for code, name in event_names.items():
    count = (train_df[event_col] == code).sum()
    print(f"  {name}: {count:,}")

In [ ]:
# Standardize features (important for neural networks)
print("=== Standardizing Features ===")

scaler = StandardScaler()

X_train = scaler.fit_transform(train_df[feature_cols]).astype('float32')
X_val = scaler.transform(val_df[feature_cols]).astype('float32')
X_test = scaler.transform(test_df[feature_cols]).astype('float32')

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

# Durations
duration_train = train_df[time_col].values
duration_val = val_df[time_col].values
duration_test = test_df[time_col].values

print(f"\nDuration range: {duration_train.min()} - {duration_train.max()} months")

---

## Fit DeepHit Models

We fit cause-specific DeepHit models for prepayment and default.

In [ ]:
# Discretize time for DeepHit
# DeepHit models discrete time, so we need to create time bins
from pycox.preprocessing.label_transforms import LabTransDiscreteTime

print("=== Discretizing Time ===")

# Create discrete time cuts based on training data
cuts = np.linspace(0, duration_train.max(), NUM_DURATIONS + 1)
labtrans = LabTransDiscreteTime(cuts)

print(f"Number of time intervals: {NUM_DURATIONS}")
print(f"Time range: {cuts[0]:.1f} - {cuts[-1]:.1f} months")
print(f"Interval width: ~{(cuts[-1] - cuts[0]) / NUM_DURATIONS:.2f} months")

In [ ]:
# Fit DeepHit for PREPAYMENT (event=1)
print("=== Fitting DeepHit for PREPAYMENT ===")

# Create cause-specific event indicator
event_prepay_train = (train_df[event_col] == 1).astype('float32').values
event_prepay_val = (val_df[event_col] == 1).astype('float32').values
event_prepay_test = (test_df[event_col] == 1).astype('float32').values

# Transform labels for DeepHit
y_train_prepay = labtrans.fit_transform(duration_train, event_prepay_train)
y_val_prepay = labtrans.transform(duration_val, event_prepay_val)

# Create network
in_features = X_train.shape[1]
out_features = labtrans.out_features

net_prepay = tt.practical.MLPVanilla(
    in_features,
    DEEPHIT_PARAMS['num_nodes'],
    out_features,
    DEEPHIT_PARAMS['batch_norm'],
    DEEPHIT_PARAMS['dropout'],
    activation=DEEPHIT_PARAMS['activation']
)

# Create model
model_prepay = DeepHitSingle(
    net_prepay,
    tt.optim.Adam,
    alpha=DEEPHIT_PARAMS['alpha'],
    sigma=DEEPHIT_PARAMS['sigma'],
    duration_index=labtrans.cuts
)
model_prepay.optimizer.set_lr(DEEPHIT_PARAMS['learning_rate'])

print(f"Network architecture: {in_features} -> {DEEPHIT_PARAMS['num_nodes']} -> {out_features}")
print(f"Training events: {event_prepay_train.sum():,.0f}")

# Fit with early stopping
callbacks = [tt.callbacks.EarlyStopping(patience=10)]
log_prepay = model_prepay.fit(
    X_train, y_train_prepay,
    DEEPHIT_PARAMS['batch_size'],
    DEEPHIT_PARAMS['epochs'],
    callbacks,
    val_data=(X_val, y_val_prepay),
    verbose=True
)

print(f"\nTraining complete. Best epoch: {log_prepay.to_pandas()['val_loss'].idxmin()}")

In [ ]:
# Fit DeepHit for DEFAULT (event=2)
print("=== Fitting DeepHit for DEFAULT ===")

# Create cause-specific event indicator
event_default_train = (train_df[event_col] == 2).astype('float32').values
event_default_val = (val_df[event_col] == 2).astype('float32').values
event_default_test = (test_df[event_col] == 2).astype('float32').values

# Transform labels for DeepHit
y_train_default = labtrans.transform(duration_train, event_default_train)
y_val_default = labtrans.transform(duration_val, event_default_val)

# Create network
net_default = tt.practical.MLPVanilla(
    in_features,
    DEEPHIT_PARAMS['num_nodes'],
    out_features,
    DEEPHIT_PARAMS['batch_norm'],
    DEEPHIT_PARAMS['dropout'],
    activation=DEEPHIT_PARAMS['activation']
)

# Create model
model_default = DeepHitSingle(
    net_default,
    tt.optim.Adam,
    alpha=DEEPHIT_PARAMS['alpha'],
    sigma=DEEPHIT_PARAMS['sigma'],
    duration_index=labtrans.cuts
)
model_default.optimizer.set_lr(DEEPHIT_PARAMS['learning_rate'])

print(f"Training events: {event_default_train.sum():,.0f}")

# Fit with early stopping
callbacks = [tt.callbacks.EarlyStopping(patience=10)]
log_default = model_default.fit(
    X_train, y_train_default,
    DEEPHIT_PARAMS['batch_size'],
    DEEPHIT_PARAMS['epochs'],
    callbacks,
    val_data=(X_val, y_val_default),
    verbose=True
)

print(f"\nTraining complete. Best epoch: {log_default.to_pandas()['val_loss'].idxmin()}")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Prepayment
ax = axes[0]
log_df = log_prepay.to_pandas()
ax.plot(log_df['train_loss'], label='Train')
ax.plot(log_df['val_loss'], label='Validation')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('DeepHit Prepayment: Training Curve')
ax.legend()
ax.grid(True, alpha=0.3)

# Default
ax = axes[1]
log_df = log_default.to_pandas()
ax.plot(log_df['train_loss'], label='Train')
ax.plot(log_df['val_loss'], label='Validation')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('DeepHit Default: Training Curve')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'deephit_training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Model Evaluation

Evaluate using time-dependent C-index at 24, 48, and 72 months.

In [ ]:
# Evaluate PREPAYMENT model
print("=== DeepHit Prepayment Model Evaluation ===")

# Get survival predictions
surv_prepay = model_prepay.predict_surv_df(X_test)

# Create EvalSurv object for pycox metrics
ev_prepay = EvalSurv(
    surv_prepay,
    duration_test,
    event_prepay_test,
    censor_surv='km'
)

# Time-dependent C-index using pycox
print("\nTime-Dependent C-index for PREPAYMENT:")
print("-" * 50)

cindex_prepay_results = {}
for tau in TIME_HORIZONS:
    try:
        c_tau = ev_prepay.concordance_td(method='antolini')
        cindex_prepay_results[tau] = c_tau
        print(f"  tau = {tau:3d} months: C-index = {c_tau:.4f}")
    except Exception as e:
        print(f"  tau = {tau:3d} months: Error - {str(e)[:50]}")

# Also compute using scikit-survival for comparison
print("\nUsing scikit-survival IPCW:")

# Get risk scores (negative mean survival = higher risk)
risk_prepay_test = -surv_prepay.mean(axis=0).values

# Create structured arrays for sksurv
y_train_prepay_sk = Surv.from_arrays(event_prepay_train.astype(bool), duration_train)
y_test_prepay_sk = Surv.from_arrays(event_prepay_test.astype(bool), duration_test)

cindex_prepay_ipcw = {}
for tau in TIME_HORIZONS:
    try:
        c_tau = concordance_index_ipcw(
            y_train_prepay_sk,
            y_test_prepay_sk,
            risk_prepay_test,
            tau=tau
        )
        cindex_prepay_ipcw[tau] = c_tau[0]
        print(f"  tau = {tau:3d} months: C-index (IPCW) = {c_tau[0]:.4f}")
    except Exception as e:
        print(f"  tau = {tau:3d} months: Error - {str(e)[:50]}")

# Overall C-index (Harrell's)
c_index_prepay = concordance_index_censored(
    event_prepay_test.astype(bool),
    duration_test,
    risk_prepay_test
)
print(f"\nOverall C-index (Harrell): {c_index_prepay[0]:.4f}")

In [ ]:
# Evaluate DEFAULT model
print("=== DeepHit Default Model Evaluation ===")

# Get survival predictions
surv_default = model_default.predict_surv_df(X_test)

# Create EvalSurv object
ev_default = EvalSurv(
    surv_default,
    duration_test,
    event_default_test,
    censor_surv='km'
)

# Time-dependent C-index
print("\nTime-Dependent C-index for DEFAULT:")
print("-" * 50)

cindex_default_results = {}
for tau in TIME_HORIZONS:
    try:
        c_tau = ev_default.concordance_td(method='antolini')
        cindex_default_results[tau] = c_tau
        print(f"  tau = {tau:3d} months: C-index = {c_tau:.4f}")
    except Exception as e:
        print(f"  tau = {tau:3d} months: Error - {str(e)[:50]}")

# Using scikit-survival IPCW
print("\nUsing scikit-survival IPCW:")

risk_default_test = -surv_default.mean(axis=0).values

y_train_default_sk = Surv.from_arrays(event_default_train.astype(bool), duration_train)
y_test_default_sk = Surv.from_arrays(event_default_test.astype(bool), duration_test)

cindex_default_ipcw = {}
for tau in TIME_HORIZONS:
    try:
        c_tau = concordance_index_ipcw(
            y_train_default_sk,
            y_test_default_sk,
            risk_default_test,
            tau=tau
        )
        cindex_default_ipcw[tau] = c_tau[0]
        print(f"  tau = {tau:3d} months: C-index (IPCW) = {c_tau[0]:.4f}")
    except Exception as e:
        print(f"  tau = {tau:3d} months: Error - {str(e)[:50]}")

# Overall C-index
c_index_default = concordance_index_censored(
    event_default_test.astype(bool),
    duration_test,
    risk_default_test
)
print(f"\nOverall C-index (Harrell): {c_index_default[0]:.4f}")

In [ ]:
# Plot time-dependent C-index comparison
fig, ax = plt.subplots(figsize=(10, 6))

# Use IPCW results for plotting
horizons = sorted(set(cindex_prepay_ipcw.keys()) & set(cindex_default_ipcw.keys()))
prepay_cindex = [cindex_prepay_ipcw[h] for h in horizons]
default_cindex = [cindex_default_ipcw[h] for h in horizons]

x = np.arange(len(horizons))
width = 0.35

bars1 = ax.bar(x - width/2, prepay_cindex, width, label='Prepayment', color='steelblue', alpha=0.8)
bars2 = ax.bar(x + width/2, default_cindex, width, label='Default', color='indianred', alpha=0.8)

# Add value labels
for bar, val in zip(bars1, prepay_cindex):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{val:.3f}', ha='center', va='bottom', fontsize=10)
for bar, val in zip(bars2, default_cindex):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{val:.3f}', ha='center', va='bottom', fontsize=10)

ax.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5, label='Random (0.5)')

ax.set_xlabel('Time Horizon (months)', fontsize=12)
ax.set_ylabel('C-index (IPCW)', fontsize=12)
ax.set_title('DeepHit: Time-Dependent Concordance Index by Event Type', fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels([f'tau = {h}' for h in horizons])
ax.set_ylim(0.4, 1.0)
ax.legend(loc='lower right')
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'deephit_time_dependent_cindex.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Figure saved to: {FIGURES_DIR / 'deephit_time_dependent_cindex.png'}")

---

## Survival Curves

Plot predicted survival curves for sample loans.

In [ ]:
# Plot survival curves for sample loans
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sample 5 random loans
np.random.seed(42)
sample_idx = np.random.choice(len(X_test), size=5, replace=False)

# Prepayment survival
ax = axes[0]
for i, idx in enumerate(sample_idx):
    surv = surv_prepay.iloc[:, idx]
    ax.plot(surv.index, surv.values, label=f'Loan {idx}', alpha=0.7)
ax.set_xlabel('Time (months)')
ax.set_ylabel('Survival Probability')
ax.set_title('DeepHit: Prepayment Survival Curves')
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

# Default survival
ax = axes[1]
for i, idx in enumerate(sample_idx):
    surv = surv_default.iloc[:, idx]
    ax.plot(surv.index, surv.values, label=f'Loan {idx}', alpha=0.7)
ax.set_xlabel('Time (months)')
ax.set_ylabel('Survival Probability')
ax.set_title('DeepHit: Default Survival Curves')
ax.legend(loc='lower left')
ax.grid(True, alpha=0.3)
ax.set_ylim(0, 1)

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'deephit_survival_curves.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Feature Importance (Permutation)

Since DeepHit is a neural network, we use permutation importance to assess feature importance.

In [ ]:
# Compute permutation importance
print("=== Feature Importance (Permutation) ===")
print("Computing permutation importance...")

def permutation_importance_deephit(model, X, duration, event, feature_names, n_repeats=10):
    """Compute permutation importance for DeepHit model."""
    # Baseline score (negative mean survival as risk score)
    surv_baseline = model.predict_surv_df(X)
    risk_baseline = -surv_baseline.mean(axis=0).values
    
    y_sk = Surv.from_arrays(event.astype(bool), duration)
    baseline_cindex = concordance_index_censored(
        event.astype(bool), duration, risk_baseline
    )[0]
    
    importances = []
    importances_std = []
    
    for i, feat in enumerate(feature_names):
        scores = []
        for _ in range(n_repeats):
            X_perm = X.copy()
            np.random.shuffle(X_perm[:, i])
            
            surv_perm = model.predict_surv_df(X_perm)
            risk_perm = -surv_perm.mean(axis=0).values
            
            perm_cindex = concordance_index_censored(
                event.astype(bool), duration, risk_perm
            )[0]
            
            scores.append(baseline_cindex - perm_cindex)
        
        importances.append(np.mean(scores))
        importances_std.append(np.std(scores))
        
    return np.array(importances), np.array(importances_std)

# Prepayment model
print("\nPrepayment model...")
imp_prepay, imp_prepay_std = permutation_importance_deephit(
    model_prepay, X_test, duration_test, event_prepay_test, feature_cols, n_repeats=5
)

importance_prepay = pd.DataFrame({
    'feature': feature_cols,
    'importance': imp_prepay,
    'std': imp_prepay_std
}).sort_values('importance', ascending=False)

# Default model
print("Default model...")
imp_default, imp_default_std = permutation_importance_deephit(
    model_default, X_test, duration_test, event_default_test, feature_cols, n_repeats=5
)

importance_default = pd.DataFrame({
    'feature': feature_cols,
    'importance': imp_default,
    'std': imp_default_std
}).sort_values('importance', ascending=False)

print("\nTop 10 Features - PREPAYMENT:")
print(importance_prepay.head(10).to_string(index=False))

print("\nTop 10 Features - DEFAULT:")
print(importance_default.head(10).to_string(index=False))

In [ ]:
# Plot feature importance comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 8))

# Prepayment
ax = axes[0]
top_n = 15
plot_df = importance_prepay.head(top_n).iloc[::-1]
ax.barh(plot_df['feature'], plot_df['importance'], xerr=plot_df['std'],
        color='steelblue', alpha=0.7, capsize=3)
ax.set_xlabel('Importance (decrease in C-index)')
ax.set_title('DeepHit Permutation Importance: Prepayment', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

# Default
ax = axes[1]
plot_df = importance_default.head(top_n).iloc[::-1]
ax.barh(plot_df['feature'], plot_df['importance'], xerr=plot_df['std'],
        color='indianred', alpha=0.7, capsize=3)
ax.set_xlabel('Importance (decrease in C-index)')
ax.set_title('DeepHit Permutation Importance: Default', fontsize=12)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig(FIGURES_DIR / 'deephit_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

---

## Save Models

In [ ]:
# Save models
model_prepay.save_net(MODELS_DIR / 'deephit_prepay_net.pt')
model_default.save_net(MODELS_DIR / 'deephit_default_net.pt')

# Save scaler and labtrans for inference
with open(MODELS_DIR / 'deephit_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open(MODELS_DIR / 'deephit_labtrans.pkl', 'wb') as f:
    pickle.dump(labtrans, f)

# Save feature importance
importance_prepay.to_csv(MODELS_DIR / 'deephit_importance_prepay.csv', index=False)
importance_default.to_csv(MODELS_DIR / 'deephit_importance_default.csv', index=False)

# Save feature columns
with open(MODELS_DIR / 'deephit_feature_cols.pkl', 'wb') as f:
    pickle.dump(feature_cols, f)

print(f"Models saved to {MODELS_DIR}:")
print(f"  - deephit_prepay_net.pt")
print(f"  - deephit_default_net.pt")
print(f"  - deephit_scaler.pkl")
print(f"  - deephit_labtrans.pkl")
print(f"  - deephit_importance_prepay.csv")
print(f"  - deephit_importance_default.csv")
print(f"  - deephit_feature_cols.pkl")

---

## Summary

In [ ]:
print("=" * 70)
print("DEEPHIT - SUMMARY")
print("=" * 70)

print(f"\nData:")
print(f"  Training loans: {len(train_df):,}")
print(f"  Validation loans: {len(val_df):,}")
print(f"  Test loans: {len(test_df):,}")

print(f"\nFeatures: {len(feature_cols)}")
print(f"  Static: {len([f for f in STATIC_FEATURES if f in feature_cols or 'log_upb' in feature_cols])}")
print(f"  Behavioral: {len([f for f in BEHAVIORAL_FEATURES if f in feature_cols or 'bal_repaid_lag1' in feature_cols])}")
print(f"  Macro: {len([f for f in MACRO_FEATURES if f in feature_cols])}")

print(f"\nDeepHit Parameters:")
for param, value in DEEPHIT_PARAMS.items():
    print(f"  {param}: {value}")

print(f"\n{'='*70}")
print("MODEL PERFORMANCE (Test Set)")
print("=" * 70)

print(f"\nPREPAYMENT MODEL:")
print(f"  Training events: {event_prepay_train.sum():,.0f}")
print(f"  Overall C-index (Harrell): {c_index_prepay[0]:.4f}")
print(f"  Time-Dependent C-index (IPCW):")
for tau, c in cindex_prepay_ipcw.items():
    print(f"    tau = {tau:3d} months: {c:.4f}")

print(f"\nDEFAULT MODEL:")
print(f"  Training events: {event_default_train.sum():,.0f}")
print(f"  Overall C-index (Harrell): {c_index_default[0]:.4f}")
print(f"  Time-Dependent C-index (IPCW):")
for tau, c in cindex_default_ipcw.items():
    print(f"    tau = {tau:3d} months: {c:.4f}")

print(f"\nTop 3 Important Features:")
print(f"  Prepayment: {', '.join(importance_prepay['feature'].head(3).tolist())}")
print(f"  Default: {', '.join(importance_default['feature'].head(3).tolist())}")

---

## Next Steps

**Notebook 09**: Model Comparison

Compare all models:
- Cause-Specific Cox (notebook 05)
- Fine-Gray (notebook 06)
- Random Survival Forest (notebook 07)
- DeepHit (this notebook)

Key comparisons:
- Time-dependent C-index at multiple horizons
- Calibration assessment
- Cumulative incidence predictions
- Computational cost